In [1]:
import torchaudio
import json
import os
from preprocess import create_reverberated_data, create_noisy_data, create_noisy_data_parallel, create_reverberated_data_parallel
from tqdm import tqdm

In [2]:
# play audio in jupyter
import IPython.display
path = "./datasets/non_concat_csj/reverberated/csj_train_nodup_sp/reverberated_sp1.1-A07F0467_1069062_1080003.wav"


IPython.display.Audio(path)


In [ ]:
# create json

data_types = ["train_nodup_sp", "train_dev", "eval1", "eval2", "eval3"]
DATA_JSON_PATH_PREFIX = "/home/shibutani/fs/ASR/espnet/egs/csj/asr1/dump/"
def get_data_json_path(data_type):
    return os.path.join(DATA_JSON_PATH_PREFIX, data_type, "deltafalse/data.json")
def get_wav_path(data_type, key):
    WAV_PATH_PREFIXS = {
        "train_nodup_sp": "/n/work3/mimura/data/csj/wav/sp",
        "train_dev": "/n/work3/mimura/data/csj/wav/wav.segments",
        "eval1": "/n/work3/mimura/data/csj/wav/wav.segments.testset",
        "eval2": "/n/work3/mimura/data/csj/wav/wav.segments.testset",
        "eval3": "/n/work3/mimura/data/csj/wav/wav.segments.testset",
    }
    wav_path_prefix = WAV_PATH_PREFIXS[data_type]
    if data_type == "train_nodup_sp":
        sp = key.split("-")[0]
        key = key.split("-")[1]
        utt_id = key.split("_")[0]
        wav_path = os.path.join(wav_path_prefix, sp, utt_id, key + ".wav")
    else:
        utt_id = key.split("_")[0]
        wav_path = os.path.join(wav_path_prefix, utt_id, key + ".wav")

    return wav_path

data_type = "train_nodup"
data_json_path = get_data_json_path(data_type)
with open(data_json_path) as f:
    data = json.load(f)
utts = data["utts"]
keys = data["utts"].keys()

result_json = {}

error_counter = 0

for key in keys:
    # show progress
    if len(result_json) % 100 == 0:
        print(len(result_json)/ len(keys) * 100, "%", end="\r")
    wav_file_path = get_wav_path(data_type, key)
    if not os.path.exists(wav_file_path):
        error_counter += 1
        continue
    sampling_rate = torchaudio.info(wav_file_path).sample_rate
    audio_sec = torchaudio.info(wav_file_path).num_frames / sampling_rate
    utt = utts[key]
    raw_transcript = ""
    for output in utt["output"]:
        raw_transcript += output["text"]
        if len(utt["output"]) > 1:
            print("Warning: utt has multiple outputs")
    result_json[key] = {}
    result_json[key]["wav_file_path"] = wav_file_path
    result_json[key]["sampling_rate"] = sampling_rate
    result_json[key]["audio_sec"] = audio_sec
    result_json[key]["raw_transcript"] = raw_transcript

print(f"error rate: {error_counter / len(keys) * 100}%")

with open(f"json/csj_{data_type}.json", "w") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

In [6]:
# create json for noise
chime3_dir_path = "/n/work3/shibutani/ASR/datasets/CHiME3/data/audio/16kHz/backgrounds"
chime3_file_paths = list(os.listdir(chime3_dir_path))
chime3_paths = [os.path.join(chime3_dir_path, path) for path in chime3_file_paths]
chime3_json = {}
for i, path in enumerate(chime3_paths):
    chime3_json[i] = {}
    chime3_json[i]["wav_file_path"] = path
    chime3_json[i]["sampling_rate"] = torchaudio.info(path).sample_rate
    chime3_json[i]["audio_sec"] = torchaudio.info(path).num_frames / torchaudio.info(path).sample_rate
    chime3_json[i]["raw_transcript"] = ""
with open("json/chime3.json", "w") as f:
    json.dump(chime3_json, f, indent=4, ensure_ascii=False)


demand_dir_paths = [os.path.join("/n/work3/mimura/data/DEMAND/demand", dir_pre) for dir_pre in os.listdir("/n/work3/mimura/data/DEMAND/demand")]
demand_paths = []
for demand_dir_path in demand_dir_paths:
    for demand_file_path in os.listdir(demand_dir_path):
        demand_paths.append(os.path.join(demand_dir_path, demand_file_path))
demand_json = {}
for i, path in enumerate(demand_paths):
    demand_json[i] = {}
    demand_json[i]["wav_file_path"] = path
    demand_json[i]["sampling_rate"] = torchaudio.info(path).sample_rate
    demand_json[i]["audio_sec"] = torchaudio.info(path).num_frames / torchaudio.info(path).sample_rate
    demand_json[i]["raw_transcript"] = ""
with open("json/demand.json", "w") as f:
    json.dump(demand_json, f, indent=4, ensure_ascii=False)

# musanは以下のファイルに記載の音声を使う
# script.noise.musan.music_noise
musan_paths_path = "/n/work3/mimura/exp/multicond/script.noise.musan.music_noise"
with open(musan_paths_path, "r") as f:
    musan_paths = f.readlines()
musan_paths = [path.strip() for path in musan_paths]
musan_json = {}
for i, path in enumerate(musan_paths):
    musan_json[i] = {}
    musan_json[i]["wav_file_path"] = path
    musan_json[i]["sampling_rate"] = torchaudio.info(path).sample_rate
    musan_json[i]["audio_sec"] = torchaudio.info(path).num_frames / torchaudio.info(path).sample_rate
    musan_json[i]["raw_transcript"] = ""
with open("json/musan.json", "w") as f:
    json.dump(musan_json, f, indent=4, ensure_ascii=False)

In [3]:
# 残響の付加 (dev, eval)
NAME = "csj_eval1"
with open(f"json/{NAME}.json", "r") as f:
    data_json = json.load(f)

result_json = create_reverberated_data(data_json, NAME)
with open(f"json/reverberated_{NAME}.json", "w") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

100%|██████████| 1272/1272 [10:21<00:00,  2.05it/s]


In [2]:
# ノイズの付加 (dev, eval)
import multiprocessing
noise_data_json = {
    "cafeteria": {
        "wav_file_path": "/n/work3/shibutani/ASR/datasets/ku-real/noise/noise_cafeteria.wav"
    },
    "museum": {
        "wav_file_path": "/n/work3/shibutani/ASR/datasets/ku-real/noise/noise_museum.wav"
    }
}
noise_data_jsons = [noise_data_json]

NAME = "csj_eval1"
with open(f"json/reverberated_{NAME}.json", "r") as f:
    data_json = json.load(f)

all_keys = list(data_json.keys())

jobs = []
queue = multiprocessing.Queue()
NUM_PROCS = 16
for i in range(NUM_PROCS):
    start = int(len(all_keys) / NUM_PROCS * i)
    end = int(len(all_keys) / NUM_PROCS * (i + 1))
    if i == NUM_PROCS - 1:
        end = len(all_keys)
    keys = all_keys[start:end]
    p = multiprocessing.Process(
        target=create_noisy_data_parallel, args=(data_json, keys, NAME, noise_data_jsons, queue)
    )
    p.start()
    jobs.append(p)

# concat result_jsons in queue
result_json = {}
for i in range(NUM_PROCS):
    result_json.update(queue.get())

for p in jobs:
    p.join()

assert len(result_json) == len(data_json)

with open(f"json/noisy_{NAME}.json", "w") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

100%|██████████| 80/80 [00:33<00:00,  2.39it/s]

100%|██████████| 80/80 [00:36<00:00,  2.21it/s]


In [4]:
# train-devに対してtrainと同様のノイズを加え、初期のパラメータチューニングに利用するデータ(development)を作成する
import multiprocessing

with open("json/chime3.json", "r") as f:
    chime3_data_json = json.load(f)
with open("json/musan.json", "r") as f:
    musan_data_json = json.load(f)
with open("json/demand.json", "r") as f:
    demand_data_json = json.load(f)

noise_data_jsons = [chime3_data_json, musan_data_json, demand_data_json]

with open(f"json/concat_reverberated_csj_train_dev.json", "r") as f:
    data_json = json.load(f)

all_keys = list(data_json.keys())

jobs = []
queue = multiprocessing.Queue()
NUM_PROCS = 16
for i in range(NUM_PROCS):
    start = int(len(all_keys) / NUM_PROCS * i)
    end = int(len(all_keys) / NUM_PROCS * (i + 1))
    if i == NUM_PROCS - 1:
        end = len(all_keys)
    keys = all_keys[start:end]
    p = multiprocessing.Process(
        target=create_noisy_data_parallel, args=(data_json, keys, "development", noise_data_jsons, queue)
    )
    p.start()
    jobs.append(p)

# concat result_jsons in queue
result_json = {}
for i in range(NUM_PROCS):
    result_json.update(queue.get())

for p in jobs:
    p.join()

assert len(result_json) == len(data_json)

with open(f"json/concat_noisy_csj_development.json", "w") as f:
    json.dump(result_json, f, indent=4, ensure_ascii=False)

100%|██████████| 69/69 [01:33<00:00,  1.36s/it]


## Tokenizer

In [8]:
# TranscriptFileの作成
with open("json/noisy_csj_train_nodup_sp.json", "r") as f:
    data_json = json.load(f)

transcripts = []
for key in tqdm(data_json.keys()):
    utt = data_json[key]
    transcript = utt["raw_transcript"]
    transcripts.append(transcript)
with open("vocabs/csj_train_nodup_sp.txt", "w") as f:
    for transcript in transcripts:
        f.write(transcript + "\n")

100%|██████████| 1158900/1158900 [00:00<00:00, 1404385.87it/s]


In [6]:
# Tokenizerの作成
from tokenizer import SentencePieceTokenizer

SentencePieceTokenizer.create_model(
    transcription_file_path="vocabs/csj_train_nodup_sp.txt",
    model_prefix="vocabs/csj_train_nodup_sp_4096.bpe",
    num_tokens=4096,
    model_type="bpe",
    character_coverage=0.9995
)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=vocabs/csj_train_nodup_sp.txt --model_prefix=vocabs/csj_train_nodup_sp_4096.bpe --vocab_size=4096 --character_coverage=0.9995 --model_type=bpe --control_symbols=<blank> --unk_id=1 --bos_id=2 --eos_id=3 --pad_id=4
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: vocabs/csj_train_nodup_sp.txt
  input_format: 
  model_prefix: vocabs/csj_train_nodup_sp_4096.bpe
  model_type: BPE
  vocab_size: 4096
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  control_symbols: <blank>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  tra

In [7]:
# Tokenizerの作成
from tokenizer import SentencePieceTokenizer

SentencePieceTokenizer.create_model(
    transcription_file_path="vocabs/yesno_train_transcripts.txt",
    model_prefix="vocabs/yesno",
    num_tokens=1000,
    model_type="char",
    character_coverage=1
)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=vocabs/yesno_train_transcripts.txt --model_prefix=vocabs/yesno --vocab_size=1000 --character_coverage=1 --model_type=char --control_symbols=<blank> --unk_id=1 --bos_id=2 --eos_id=3 --pad_id=4
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: vocabs/yesno_train_transcripts.txt
  input_format: 
  model_prefix: vocabs/yesno
  model_type: CHAR
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  control_symbols: <blank>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_